In [12]:
# Magic + imports likely common across all notebooks
%load_ext autoreload
%autoreload 2
%matplotlib inline
# Supress Warning 
import warnings
warnings.filterwarnings('ignore')
# Set reference for util modules
import sys
sys.path.append('/home/jovyan/odc-hub/')
# Generic python
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr 
from datetime import datetime

# sac-specific
from sac_utils.createAOI import create_lat_lon

# dc-specific
from datacube import Datacube
from datacube.storage.masking import mask_invalid_data
dc = Datacube()

# ml stuff
from sklearn.ensemble import RandomForestClassifier

import utils_dcal.data_cube_utilities.data_access_api as dc_api
api = dc_api.DataAccessApi()

# dask stuff
import dask.array
import dask
from dask.distributed import Client
client = Client('dask-scheduler.dask.svc.cluster.local:8786')
client

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Client Scheduler: tcp://dask-scheduler.dask.svc.cluster.local:8786 Dashboard: http://dask-scheduler.dask.svc.cluster.local:8787/status,Cluster Workers: 5 Cores: 40 Memory: 168.65 GB


In [13]:
dc.find_datasets(product='s1g_gamma0_scene',
                 lat=latitude,
                 lon=longitude,)

[]

In [14]:
dc.find_datasets(product='s1g_gamma0_scene')[0].extent.envelope, dc.find_datasets(product='s1g_gamma0_scene')[0].crs.epsg

(BoundingBox(left=176.18476151990743, bottom=-19.706139263944557, right=179.03978715589608, top=-17.04191579430289),
 4326)

In [15]:
dc.list_measurements('s1g_gamma0_scene')

aliases  \
product                  measurement                                                       
ls4_usgs_sr_scene        blue                                         [band_1, sr_band1]   
                         green                                        [band_2, sr_band2]   
                         red                                          [band_3, sr_band3]   
                         nir                                          [band_4, sr_band4]   
                         swir1                                        [band_5, sr_band5]   
                         swir2                                        [band_7, sr_band7]   
                         sr_atmos_opacity                                     [atmos_op]   
                         pixel_qa                                             [pixel_qa]   
                         radsat_qa                                           [radsat_qa]   
                         sr_cloud_qa                                          [pixel_qa]   
ls4_water_classification water                              [wofs, water_classification]   
ls5_usgs_sr_scene        blue                                         [band_1, sr_band1]   
                         green                                        [band_2, sr_band2]   
                         red                                          [band_3, sr_band3]   
                         nir                                          [band_4, sr_band4]   
                         swir1                                        [band_5, sr_band5]   
                         swir2                                        [band_7, sr_band7]   
                         sr_atmos_opacity                                     [atmos_op]   
                         pixel_qa                                             [pixel_qa]   
                         radsat_qa                                           [radsat_qa]   
                         sr_cloud_qa                                          [pixel_qa]   
ls5_water_classification water                              [wofs, water_classification]   
ls7_usgs_sr_scene        blue                                         [band_1, sr_band1]   
                         green                                        [band_2, sr_band2]   
                         red                                          [band_3, sr_band3]   
                         nir                                          [band_4, sr_band4]   
                         swir1                                        [band_5, sr_band5]   
                         swir2                                        [band_7, sr_band7]   
                         sr_atmos_opacity                                     [atmos_op]   
                         pixel_qa                                             [pixel_qa]   
...                                                                                  ...   
ls8_usgs_sr_scene        blue                                         [band_2, sr_band2]   
                         green                                        [band_3, sr_band3]   
                         red                                          [band_4, sr_band4]   
                         nir                                          [band_5, sr_band5]   
                         swir1                                        [band_6, sr_band6]   
                         swir2                                        [band_7, sr_band7]   
                         pixel_qa                                             [pixel_qa]   
                         sr_aerosol                 [sr_aerosol_qa, aerosol_qa, aerosol]   
                         radsat_qa                                           [radsat_qa]   
                         brightness_temperature_1                   [band_10, bt_band10]   
                         brightness_temperature_2                   [band_11, bt_band11]   
ls8_water_classification water   

In [29]:
# arbitrary small-ish aoi (over suva)
aoi_wkt = "POLYGON((177.39443449660928 -17.388751818798674,178.48070768996865 -17.388751818798674,178.48070768996865 -18.170096116362053,177.39443449660928 -18.170096116362053,177.39443449660928 -17.388751818798674))"
# resulution
res = 90

output_crs = "EPSG:4326"
latitude, longitude = create_lat_lon(aoi_wkt)
# latitude = (-18.2962, -17.2867)
# longitude = (177.1594, 178.7222)
resolution = (-res, res)

In [34]:
product = 's1g_gamma0_scene'
measurements = ['vv','vh']

water_dataset = dc.load(
    product=product,
#     time=T1, 
    lat=latitude,
    lon=longitude,
    output_crs=output_crs,
    resolution=resolution,
    measurements = measurements,
    group_by='solar_day',
    dask_chunks={
        #'time': 1,
        'x': 1000,
        'y': 1000,
    }
)
water_dataset

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*

In [19]:
water_dataset.extent

Geometry(POLYGON ((413190 8115930,413190 7819740,717300 7819740,717300 8115930,413190 8115930)), EPSG:32760)

In [12]:
# Identify pixels with valid data
good_quality = (
    (water_dataset.vv != 0) # no data
)
# Apply mask
water_dataset = water_dataset.where(good_quality)

In [ ]:
water_dataset.isel(time=0).vv.plot()